In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
# from sklearn.utils import resample

train_df = pd.read_feather("/content/drive/MyDrive/data_ftr/train_data.ftr")
# test_df = pd.read_feather("/content/drive/MyDrive/data_ftr/test_data.ftr")

In [14]:
train_df.shape

(5531451, 191)

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: category(11), datetime64[ns](1), float16(177), int64(1), object(1)
memory usage: 2.0+ GB


In [16]:
train_df = train_df.groupby('customer_ID')
train_df = train_df.tail(1)

In [17]:
train_df.shape

(458913, 191)

In [18]:
# customer_ID, S_2 컬럼 제거 (유의미 하지 않음)
train_df = train_df.drop(['customer_ID', 'S_2'], axis=1)

In [19]:
train_df.head(30)

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
12,0.934570,0.009117,0.009384,1.007812,0.006104,0.135010,0.001604,0.007175,NaN,NaN,...,NaN,NaN,0.007187,0.004234,0.005085,NaN,0.005810,0.002970,0.008530,0
25,0.880371,0.178101,0.034698,1.003906,0.006912,0.165527,0.005550,0.005070,NaN,0.060638,...,NaN,NaN,0.002981,0.007481,0.007874,NaN,0.003284,0.003170,0.008514,0
38,0.880859,0.009705,0.004284,0.812500,0.006451,NaN,0.003796,0.007195,NaN,NaN,...,NaN,NaN,0.007381,0.006622,0.000965,NaN,0.002201,0.000834,0.003445,0
51,0.621582,0.001082,0.012566,1.005859,0.007828,0.287842,0.004532,0.009941,NaN,0.046112,...,NaN,NaN,0.002705,0.006184,0.001899,NaN,0.008186,0.005558,0.002983,0
64,0.872070,0.005573,0.007679,0.815918,0.001247,NaN,0.000231,0.005527,NaN,0.044678,...,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008156,0.006943,0.000905,0
77,0.824219,0.009773,0.007851,1.001953,0.006886,0.395752,0.002855,0.006134,NaN,0.013878,...,NaN,NaN,0.009399,0.007835,0.000957,NaN,0.009056,0.006943,0.001446,0
90,0.477051,0.000467,0.009415,1.008789,0.007774,0.267090,0.006126,0.125977,NaN,0.057739,...,NaN,NaN,0.006012,0.005222,0.000937,NaN,0.009789,0.003704,0.006275,0
103,1.009766,0.006180,-0.000886,1.006836,0.006111,0.116394,0.000824,0.149536,NaN,0.014717,...,NaN,NaN,0.002262,0.005219,0.002153,NaN,0.001169,0.003349,0.000675,0
116,0.424561,0.531250,0.979492,0.029297,0.008499,0.152588,0.005344,1.156250,NaN,NaN,...,NaN,NaN,1.000977,0.007061,0.875977,0.184570,1.004883,0.003349,0.734375,1
129,0.359863,0.009804,0.090698,0.818359,0.507324,0.182007,0.002934,0.009865,NaN,0.246704,...,NaN,NaN,0.009811,0.002213,0.004070,NaN,0.005428,0.007107,0.006271,1


# Top 30 Features for XGBoost

In [20]:
train_df = train_df[["P_2", "D_43", "P_3", "S_3", "D_39", "D_46", "B_4", "B_3", "B_5", "D_41", "D_47", "D_44", "D_42", "R_1", "S_9", "D_45", "D_122", "B_17", "D_55", "S_7", "R_3", "D_70", "R_5", "D_59", "R_11", "D_56", "D_48", "S_26", "D_91", "D_52", "target"]]

In [21]:
train_df.shape

(458913, 31)

In [22]:
train_df.head(30)

,P_2,D_43,P_3,S_3,D_39,D_46,B_4,B_3,B_5,D_41,...,D_70,R_5,D_59,R_11,D_56,D_48,S_26,D_91,D_52,target
12,0.934570,NaN,0.629395,0.135010,0.009117,0.420410,0.070801,0.007175,0.231689,0.001604,...,0.008255,0.009445,0.063477,0.009354,0.166626,0.192383,0.001281,1.005859,0.203491,0
25,0.880371,0.060638,0.570801,0.165527,0.178101,0.438721,0.020630,0.005070,0.026993,0.005550,...,0.000883,0.005878,0.212524,0.009666,0.748535,0.014694,0.003212,0.009163,0.242310,0
38,0.880859,NaN,0.628906,NaN,0.009705,0.433594,0.031052,0.007195,0.001557,0.003796,...,0.006100,0.003000,0.213013,0.005348,0.209351,0.080383,0.004704,0.009407,0.202148,0
51,0.621582,0.046112,0.671875,0.287842,0.001082,0.410645,0.007236,0.009941,0.118835,0.004532,...,0.006016,0.001347,0.509277,0.507324,0.554688,NaN,0.043701,0.509766,0.198364,0
64,0.872070,0.044678,0.570312,NaN,0.005573,0.465576,0.269287,0.005527,0.004856,0.000231,...,0.001121,0.008858,0.488770,0.009087,0.183105,0.325195,0.002346,0.006962,0.253906,0
77,0.824219,0.013878,0.684570,0.395752,0.009773,NaN,0.004433,0.006134,0.012749,0.002855,...,0.005905,0.001231,0.006470,0.002764,NaN,NaN,0.004669,0.004375,0.033936,0
90,0.477051,0.057739,0.714844,0.267090,0.000467,NaN,0.002541,0.125977,0.006760,0.006126,...,0.000439,0.002060,0.480469,0.007797,NaN,0.676758,0.002533,0.008987,0.030014,0
103,1.009766,0.014717,0.609863,0.116394,0.006180,0.503418,0.051331,0.149536,0.063904,0.000824,...,0.253418,0.003620,0.357910,0.003794,0.250732,0.110229,0.008774,0.000199,0.272705,0
116,0.424561,NaN,0.620605,0.152588,0.531250,0.524414,1.105469,1.156250,0.025940,0.005344,...,0.506836,0.004116,0.544922,0.005627,NaN,0.863281,0.005772,0.003063,0.081360,1
129,0.359863,0.246704,0.374756,0.182007,0.009804,0.731934,0.631836,0.009865,0.002905,0.002934,...,0.750488,2.001953,0.543945,0.002884,0.138306,0.973633,0.001798,0.001164,0.148682,1


# missing values

In [23]:
# 각 컬럼별 결측치 확인
train_df.isnull().sum()

P_2         2969
D_43      134322
P_3        22220
S_3        84970
D_39           0
D_46       95123
B_4            0
B_3           31
B_5            0
D_41          31
D_47           0
D_44       22295
D_42      399003
R_1            0
S_9       183858
D_45          31
D_122       6017
B_17      244471
D_55       30377
S_7        84970
R_3            0
D_70        4196
R_5            0
D_59        4086
R_11           0
D_56      244734
D_48       57992
S_26           0
D_91       12807
D_52        1240
target         0
dtype: int64

In [24]:
null = round((train_df.isna().sum()/train_df.shape[0]*100),2).sort_values(ascending=False).astype(str)+('%')
null = null.to_frame().rename(columns={0:'Missing %'})
null.head(30)

,Missing %
D_42,86.95%
D_56,53.33%
B_17,53.27%
S_9,40.06%
D_43,29.27%
D_46,20.73%
S_3,18.52%
S_7,18.52%
D_48,12.64%
D_55,6.62%


# Data Split

In [25]:
X = train_df.drop('target', axis=1)
y = train_df['target']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
X_train.shape

(367130, 30)

In [27]:
X_test.shape

(91783, 30)

In [28]:
y_train.shape

(367130,)

In [29]:
y_test.shape

(91783,)

# Model Training

In [30]:
# XGBoost 모델 학습 (eval_set을 사용하여 검증 데이터를 모니터링)
model = XGBClassifier(n_estimators=200, learning_rate=0.15, subsample=0.5)
model.fit(X_train, y_train)

# pkl 파일 추출
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [31]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()

    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

# 예측 확률 계산
pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = pd.DataFrame({'prediction': pred_prob}, index=X_test.index)
y_true = pd.DataFrame({'target': y_test}, index=X_test.index)

# 성능 평가
print("AMEX Score:", amex_metric(y_true, y_pred))

AMEX Score: 0.7640096176047383


In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델의 성능 리포트를 출력하는 함수
def generate_model_report(y_true, y_pred):
    # 평가 지표 계산
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # 결과 출력
    print("Model Performance:")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}")

# 예측 결과와 실제 타겟 데이터를 준비
y_true = y_test
y_pred = model.predict(X_test)

# 모델의 성능 리포트 출력
generate_model_report(y_true, y_pred)

Model Performance:
MSE: 0.10510661015656494
MAE: 0.10510661015656494
R^2: 0.4488704381880073


In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)

# 정밀도 계산
precision = precision_score(y_test, y_pred)

# 재현율 계산
recall = recall_score(y_test, y_pred)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# 분류 보고서 출력
report = classification_report(y_test, y_pred)
print("\nXGBoost Report:\n", report)

Accuracy: 0.894893389843435
Precision: 0.7953077184631078
Precision: 0.7953077184631078
Recall: 0.7948010024211019
F1 Score: 0.7950542797051263

XGBoost Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     68240
           1       0.80      0.79      0.80     23543

    accuracy                           0.89     91783
   macro avg       0.86      0.86      0.86     91783
weighted avg       0.89      0.89      0.89     91783



In [34]:
# test_data 로드
test_df = pd.read_feather("/content/drive/MyDrive/data_ftr/test_data.ftr")

In [35]:
test_df = test_df.groupby('customer_ID')
test_df = test_df.tail(1)
test_df.shape

(924621, 190)

In [37]:
# customer_ID 따로 저장
customer_ID = test_df['customer_ID']

In [39]:
test_df = test_df[["P_2", "D_43", "P_3", "S_3", "D_39", "D_46", "B_4", "B_3", "B_5", "D_41", "D_47", "D_44", "D_42", "R_1", "S_9", "D_45", "D_122", "B_17", "D_55", "S_7", "R_3", "D_70", "R_5", "D_59", "R_11", "D_56", "D_48", "S_26", "D_91", "D_52"]]

In [40]:
# XGBoostClassifier 모델로 예측
pred = model.predict_proba(test_df)[:, 1]

In [41]:
# customer_ID와 prediction을 submission.csv로 추출
submission = pd.DataFrame({'customer_ID': customer_ID, 'prediction': pred})
submission.to_csv('submission_predict(Top 30 Feature Importance).csv', index=False)